In [1]:
# Import Python package
import BinhoSupernova
from BinhoSupernova.Supernova import Supernova
from BinhoSupernova.commands.definitions import *


## Getting started

#### 1. List all the Supernova devices connected to the PC host

The ``BinhoSupernova.getConnectedSupernovaDevicesList()`` gets a list of the Supernova devices plugged into the host PC machine.

In [2]:
BinhoSupernova.getConnectedSupernovaDevicesList()

[{'path': '\\\\?\\HID#VID_1FC9&PID_82FC#7&f321146&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}',
  'vendor_id': '0x1fc9',
  'product_id': '0x82fc',
  'serial_number': '59314AC593A1C05C8163B6759994476',
  'release_number': 256,
  'manufacturer_string': 'Binho LLC',
  'product_string': 'Binho Supernova',
  'usage_page': 65280,
  'usage': 1,
  'interface_number': -1}]

#### 2. Create an instance of the Supernova class

To utilize a Supernova USB host adapter device, we need to create an instance of the Supernova class.

In [3]:
# Create a device instance. One instance of the Supernova class represents a Supernova USB host adapter device.
supernova = Supernova()

#### 3. Open connection to the Supernova device

The public method ``Supernova.open()`` establishes the connection with a Supernova device. Below is the complete signature:

```python
open(vid, pid, serial, path)
```

- ``vid: int``: The Supernova USB VID, which is set by default.
- ``pid: int``: The Supernova USB PID, which is set by default.
- ``serial: str``: The Supernova serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``BinhoSupernova.getConnectedSupernovaDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Supernova device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Supernova device simultaneously.


In [4]:
# Use the method by default to connect to only one Supernova device.
supernova.open()

# Otherwise, use the path attribute to identify each Supernova device. Uncomment the line below and comment the line #2.
# supernova.open(path='\\\\?\\HID#VID_1FC9&PID_82FC#6&48d9417&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}')

{'module': 0,
 'opcode': 0,
 'message': 'Connection with Supernova device opened successfully.'}

#### 4. Define and register a callback to handle responses and notifications from Supernova

To handle responses and notifications from Supernova, a callback function must be defined and registered. This function will be invoked every time the Supernova sends a response to a request, an asynchronous notification, or a message from the system.

The callback function's signature is as follows: 

``def callback_function_name(supernova_message: dict, system_message: dict) -> None:``

Once the callback function is defined, it should be registered using the ``Supernova.onEvent(callback_function)`` method.

In [5]:
# Define callback function
def callback_function(supernova_message: dict, system_message: dict) -> None:
    
    if supernova_message != None:

        # Print mesasage
        print(">> New message from SUPERNOVA:")
        print(supernova_message)
    
    if system_message != None:

        # Print message
        print(">> New message from the SYSTEM:")
        print(system_message)

In [6]:
# Register callback function
supernova.onEvent(callback_function)

{'module': 0,
 'opcode': 0,
 'message': 'On event callback function registered successfully.'}

#### 5. Define a function to generate transaction IDs

All the request messages sent to the Supernova from the USB Host application must include the transaction or request ID. The ID is a 2-byte integer with an allowed range of ``[0, 65535]``.

In this example, a dummy function called ``getId()`` is defined to increment a transaction counter used as the ID.

In [7]:
#Auxiliar code to generate IDs.

counter_id = 0

def getId():
    global counter_id
    counter_id = counter_id + 1
    return counter_id

## Using the UART peripheral  

### 1. Connecting TX to RX for UART Loopback Test

To perform the UART test:

**Connection Setup:**
   - Physically connect the TX (transmit) pin to the RX (receive) pin on your device.
   - This direct connection forms a loopback, allowing transmitted data to be immediately received back by the device.

**Expected Messages and Notifications:**
   - Upon writing data:
     - Expect a confirmation of the write status.
     - If the write is successful, anticipate a notification containing the transmitted data.
   
**Testing Process:**
   - Write data using the Supernovas's functionalities.
   - Look for immediate feedback:
     - Confirmation message indicating the write status.
     - Notification displaying the transmitted data if the write operation succeeds.



### 2. Initializing the UART Peripheral

To initialize the UART peripheral:

**Setting UART Peripheral Voltage:**
   - Before initializing, ensure the UART peripheral voltage is appropriately configured (3.3V in this case).
   - Proper voltage settings are crucial for the correct operation of the UART communication.

In [8]:
supernova.setI2cSpiUartBusVoltage(getId(),3300)

{'module': 1,
 'opcode': 0,
 'message': 'SET I2C SPI UART BUS VOLTAGE requests success'}

>> New message from SUPERNOVA:
{'id': 1, 'command': 100, 'name': 'SET I2C-SPI-UART BUS VOLTAGE', 'result': 0}


**Initialize UART:**

When initializing the UART peripheral, consider the following configurable parameters:

* _Baudrate:_  
   - Defines the speed of data transmission, ranging from 600 bps to 115200 bps.  

* _Hardware Handshake (currently disabled):_ 
   - Enables or disables the Request to Send/Clear to Send (RTS/CTS) signal for flow control.  

* _Parity Mode:_ 
   - Refers to the parity check used in UART packages, offering error detection.

* _Data Character Size:_ 
   - Determines the length of each UART data package in bits.
   
* _Stop Bit:_  
   - Specifies the number of stop bits used at the end of each UART transaction.

For this case we are using:

* Baudrate: 600bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data size
* Stop Bit: single stop bit (1 bit stop)  



In [9]:
supernova.uartControllerInit(getId(),UartControllerBaudRate.UART_BAUD_600, False, UartControllerParity.UART_NO_PARITY, UartControllerDataSize.UART_8BIT_BYTE,UartControllerStopBit.UART_ONE_STOP_BIT)

{'module': 1, 'opcode': 0, 'message': 'UART CONTROLLER INIT requests success'}

>> New message from SUPERNOVA:
{'id': 2, 'command': 65, 'name': 'UART CONTROLLER INIT', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_ALREADY_INITIALIZED_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}


### 3. Sending Data via UART

The Supernova system can transmit data via UART, the data to be transmitted must adhere to specific criteria:

**Data Format:**
   - The data format consists of an array with a length less than 1024.
   - Each element in the array represents a byte value, constrained within the range of 0x00 to 0xFF.

For this test an array with 1024 elements, where each value stays within the byte range by using the modulo operation (`%`) with `0xFF` is used.


In [10]:
data = [i%0xFF for i in range(0,1024)]

In [11]:
supernova.uartControllerSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE requests success'}

>> New message from SUPERNOVA:
{'id': 3, 'command': 67, 'name': ' UART CONTROLLER SEND MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}
>> New message from SUPERNOVA:
{'id': 0, 'command': 68, 'name': ' UART CONTROLLER RECEIVE MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR', 'payload_length': 1024, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134

## Change the UART peripheral parameters

Once the UART peripheral is initialized, adjustments to its configuration can be made using the _uartControllerSetParameters_ function. This function allows modification of the following parameters, similar to the initialization:

* _Baudrate:_  
   - Defines the speed of data transmission, ranging from 600 bps to 115200 bps.  

* _Hardware Handshake (currently disabled):_ 
   - Enables or disables the Request to Send/Clear to Send (RTS/CTS) signal for flow control.  

* _Parity Mode:_ 
   - Refers to the parity check used in UART packages, offering error detection.

* _Data Character Size:_ 
   - Determines the length of each UART data package in bits.
   
* _Stop Bit:_  
   - Specifies the number of stop bits used at the end of each UART transaction.


### Baudrate 1200 bps

For this case we are using:

* Baudrate: 1200 bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data
* Stop Bit: 1 bit stop 


In [12]:
supernova.uartControllerSetParameters(getId(),UartControllerBaudRate.UART_BAUD_1200,False,UartControllerParity.UART_NO_PARITY,UartControllerDataSize.UART_8BIT_BYTE,UartControllerStopBit.UART_ONE_STOP_BIT)

{'module': 1,
 'opcode': 0,
 'message': 'UART CONTROLLER SET PARAMETERS requests success'}

>> New message from SUPERNOVA:
{'id': 4, 'command': 66, 'name': ' UART CONTROLLER SET PARAMETERS', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}


In [13]:
supernova.uartControllerSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE requests success'}

>> New message from SUPERNOVA:
{'id': 5, 'command': 67, 'name': ' UART CONTROLLER SEND MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}
>> New message from SUPERNOVA:
{'id': 0, 'command': 68, 'name': ' UART CONTROLLER RECEIVE MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR', 'payload_length': 1024, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134

### Baudrate 2400 bps

For this case we are using:

* Baudrate: 2400 bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data
* Stop Bit: 1 bit stop 

In [14]:
supernova.uartControllerSetParameters(getId(),UartControllerBaudRate.UART_BAUD_2400,False,UartControllerParity.UART_NO_PARITY,UartControllerDataSize.UART_8BIT_BYTE,UartControllerStopBit.UART_ONE_STOP_BIT)

{'module': 1,
 'opcode': 0,
 'message': 'UART CONTROLLER SET PARAMETERS requests success'}

>> New message from SUPERNOVA:
{'id': 6, 'command': 66, 'name': ' UART CONTROLLER SET PARAMETERS', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}


In [15]:
supernova.uartControllerSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE requests success'}

>> New message from SUPERNOVA:
{'id': 7, 'command': 67, 'name': ' UART CONTROLLER SEND MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}
>> New message from SUPERNOVA:
{'id': 0, 'command': 68, 'name': ' UART CONTROLLER RECEIVE MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR', 'payload_length': 1024, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134

### Baudrate 4800 bps

For this case we are using:

* Baudrate: 4800 bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data 
* Stop Bit: 1 bit stop 

In [16]:
supernova.uartControllerSetParameters(getId(),UartControllerBaudRate.UART_BAUD_4800,False,UartControllerParity.UART_NO_PARITY,UartControllerDataSize.UART_8BIT_BYTE,UartControllerStopBit.UART_ONE_STOP_BIT)

{'module': 1,
 'opcode': 0,
 'message': 'UART CONTROLLER SET PARAMETERS requests success'}

>> New message from SUPERNOVA:
{'id': 8, 'command': 66, 'name': ' UART CONTROLLER SET PARAMETERS', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}


In [17]:
supernova.uartControllerSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE requests success'}

>> New message from SUPERNOVA:
{'id': 9, 'command': 67, 'name': ' UART CONTROLLER SEND MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}
>> New message from SUPERNOVA:
{'id': 0, 'command': 68, 'name': ' UART CONTROLLER RECEIVE MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR', 'payload_length': 1024, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134

### Baudrate 9600 bps

For this case we are using:

* Baudrate: 9600 bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data
* Stop Bit: 1 bit stop 

In [18]:
supernova.uartControllerSetParameters(getId(),UartControllerBaudRate.UART_BAUD_9600,False,UartControllerParity.UART_NO_PARITY,UartControllerDataSize.UART_8BIT_BYTE,UartControllerStopBit.UART_ONE_STOP_BIT)

{'module': 1,
 'opcode': 0,
 'message': 'UART CONTROLLER SET PARAMETERS requests success'}

>> New message from SUPERNOVA:
{'id': 10, 'command': 66, 'name': ' UART CONTROLLER SET PARAMETERS', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}


In [19]:
supernova.uartControllerSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE requests success'}

>> New message from SUPERNOVA:
{'id': 11, 'command': 67, 'name': ' UART CONTROLLER SEND MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}
>> New message from SUPERNOVA:
{'id': 0, 'command': 68, 'name': ' UART CONTROLLER RECEIVE MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR', 'payload_length': 1024, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 13

### Baudrate 14400 bps

For this case we are using:

* Baudrate: 14400 bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data
* Stop Bit: 1 bit stop 

In [20]:
supernova.uartControllerSetParameters(getId(),UartControllerBaudRate.UART_BAUD_14400,False,UartControllerParity.UART_NO_PARITY,UartControllerDataSize.UART_8BIT_BYTE,UartControllerStopBit.UART_ONE_STOP_BIT)

{'module': 1,
 'opcode': 0,
 'message': 'UART CONTROLLER SET PARAMETERS requests success'}

>> New message from SUPERNOVA:
{'id': 12, 'command': 66, 'name': ' UART CONTROLLER SET PARAMETERS', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}


In [21]:
supernova.uartControllerSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE requests success'}

>> New message from SUPERNOVA:
{'id': 13, 'command': 67, 'name': ' UART CONTROLLER SEND MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}
>> New message from SUPERNOVA:
{'id': 0, 'command': 68, 'name': ' UART CONTROLLER RECEIVE MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR', 'payload_length': 1024, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 13

### Baudrate 19200 bps

For this case we are using:

* Baudrate: 19200 bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data 
* Stop Bit: 1 bit stop 

In [22]:
supernova.uartControllerSetParameters(getId(),UartControllerBaudRate.UART_BAUD_19200,False,UartControllerParity.UART_NO_PARITY,UartControllerDataSize.UART_8BIT_BYTE,UartControllerStopBit.UART_ONE_STOP_BIT)

{'module': 1,
 'opcode': 0,
 'message': 'UART CONTROLLER SET PARAMETERS requests success'}

>> New message from SUPERNOVA:
{'id': 14, 'command': 66, 'name': ' UART CONTROLLER SET PARAMETERS', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}


In [23]:
supernova.uartControllerSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE requests success'}

>> New message from SUPERNOVA:
{'id': 15, 'command': 67, 'name': ' UART CONTROLLER SEND MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}
>> New message from SUPERNOVA:
{'id': 0, 'command': 68, 'name': ' UART CONTROLLER RECEIVE MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR', 'payload_length': 1024, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 13

### Baudrate 38400 bps

For this case we are using:

* Baudrate: 38400 bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data
* Stop Bit: 1 bit stop 

In [24]:
supernova.uartControllerSetParameters(getId(),UartControllerBaudRate.UART_BAUD_38400,False,UartControllerParity.UART_NO_PARITY,UartControllerDataSize.UART_8BIT_BYTE,UartControllerStopBit.UART_ONE_STOP_BIT)

{'module': 1,
 'opcode': 0,
 'message': 'UART CONTROLLER SET PARAMETERS requests success'}

>> New message from SUPERNOVA:
{'id': 16, 'command': 66, 'name': ' UART CONTROLLER SET PARAMETERS', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}


In [25]:
supernova.uartControllerSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE requests success'}

>> New message from SUPERNOVA:
{'id': 17, 'command': 67, 'name': ' UART CONTROLLER SEND MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}
>> New message from SUPERNOVA:
{'id': 0, 'command': 68, 'name': ' UART CONTROLLER RECEIVE MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR', 'payload_length': 1024, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 13

### Baudrate 56000 bps

For this case we are using:

* Baudrate: 56000 bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data
* Stop Bit: 1 bit stop 

In [26]:
supernova.uartControllerSetParameters(getId(),UartControllerBaudRate.UART_BAUD_56000,False,UartControllerParity.UART_NO_PARITY,UartControllerDataSize.UART_8BIT_BYTE,UartControllerStopBit.UART_ONE_STOP_BIT)

{'module': 1,
 'opcode': 0,
 'message': 'UART CONTROLLER SET PARAMETERS requests success'}

>> New message from SUPERNOVA:
{'id': 18, 'command': 66, 'name': ' UART CONTROLLER SET PARAMETERS', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}


In [27]:
supernova.uartControllerSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE requests success'}

>> New message from SUPERNOVA:
{'id': 19, 'command': 67, 'name': ' UART CONTROLLER SEND MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}
>> New message from SUPERNOVA:
{'id': 0, 'command': 68, 'name': ' UART CONTROLLER RECEIVE MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR', 'payload_length': 1024, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 13

### Baudrate 57600 bps

For this case we are using:

* Baudrate: 57600 bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data
* Stop Bit: 1 bit stop 

In [28]:
supernova.uartControllerSetParameters(getId(),UartControllerBaudRate.UART_BAUD_57600,False,UartControllerParity.UART_NO_PARITY,UartControllerDataSize.UART_8BIT_BYTE,UartControllerStopBit.UART_ONE_STOP_BIT)

{'module': 1,
 'opcode': 0,
 'message': 'UART CONTROLLER SET PARAMETERS requests success'}

>> New message from SUPERNOVA:
{'id': 20, 'command': 66, 'name': ' UART CONTROLLER SET PARAMETERS', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}


In [29]:
supernova.uartControllerSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE requests success'}

>> New message from SUPERNOVA:
{'id': 21, 'command': 67, 'name': ' UART CONTROLLER SEND MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}
>> New message from SUPERNOVA:
{'id': 0, 'command': 68, 'name': ' UART CONTROLLER RECEIVE MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR', 'payload_length': 1024, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 13

### Baudrate 115200 bps

For this case we are using:

* Baudrate: 115200 bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data
* Stop Bit: 1 bit stop 

In [30]:
supernova.uartControllerSetParameters(getId(),UartControllerBaudRate.UART_BAUD_115200,False,UartControllerParity.UART_NO_PARITY,UartControllerDataSize.UART_8BIT_BYTE,UartControllerStopBit.UART_ONE_STOP_BIT)

{'module': 1,
 'opcode': 0,
 'message': 'UART CONTROLLER SET PARAMETERS requests success'}

>> New message from SUPERNOVA:
{'id': 22, 'command': 66, 'name': ' UART CONTROLLER SET PARAMETERS', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}


In [31]:
supernova.uartControllerSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE requests success'}

>> New message from SUPERNOVA:
{'id': 23, 'command': 67, 'name': ' UART CONTROLLER SEND MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}
>> New message from SUPERNOVA:
{'id': 0, 'command': 68, 'name': ' UART CONTROLLER RECEIVE MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR', 'payload_length': 1024, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 13

### Baudrate 115200 bps and Even Parity

For this case we are using:

* Baudrate: 115200 bps
* Hardware handshake: False
* Parity Mode: Even
* Data size: 8 bit data 
* Stop Bit: 1 bit stop 

In [32]:
supernova.uartControllerSetParameters(getId(),UartControllerBaudRate.UART_BAUD_115200,False,UartControllerParity.UART_EVEN_PARITY,UartControllerDataSize.UART_8BIT_BYTE,UartControllerStopBit.UART_ONE_STOP_BIT)

{'module': 1,
 'opcode': 0,
 'message': 'UART CONTROLLER SET PARAMETERS requests success'}

>> New message from SUPERNOVA:
{'id': 24, 'command': 66, 'name': ' UART CONTROLLER SET PARAMETERS', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}


In [33]:
supernova.uartControllerSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE requests success'}

>> New message from SUPERNOVA:
{'id': 25, 'command': 67, 'name': ' UART CONTROLLER SEND MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}
>> New message from SUPERNOVA:
{'id': 0, 'command': 68, 'name': ' UART CONTROLLER RECEIVE MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR', 'payload_length': 1024, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 13

### Baudrate 115200 bps, Even Parity and 2 bit stop

For this case we are using:

* Baudrate: 115200 bps
* Hardware handshake: False
* Parity Mode: Even
* Data size: 8 bit data 
* Stop Bit: 2 bit stop 

In [34]:
supernova.uartControllerSetParameters(getId(),UartControllerBaudRate.UART_BAUD_115200,False,UartControllerParity.UART_EVEN_PARITY,UartControllerDataSize.UART_8BIT_BYTE,UartControllerStopBit.UART_TWO_STOP_BIT)

{'module': 1,
 'opcode': 0,
 'message': 'UART CONTROLLER SET PARAMETERS requests success'}

>> New message from SUPERNOVA:
{'id': 26, 'command': 66, 'name': ' UART CONTROLLER SET PARAMETERS', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}


In [35]:
supernova.uartControllerSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE requests success'}

>> New message from SUPERNOVA:
{'id': 27, 'command': 67, 'name': ' UART CONTROLLER SEND MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}
>> New message from SUPERNOVA:
{'id': 0, 'command': 68, 'name': ' UART CONTROLLER RECEIVE MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR', 'payload_length': 1024, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 13

### Baudrate 115200 bps, Even Parity, 7 bit data character and 2 stop bit

For this case we are using:

* Baudrate: 115200 bps
* Hardware handshake: False
* Parity Mode: Even
* Data size: 7 bit data 
* Stop Bit: 2 bit stop 

In [36]:
supernova.uartControllerSetParameters(getId(),UartControllerBaudRate.UART_BAUD_115200,False,UartControllerParity.UART_EVEN_PARITY,UartControllerDataSize.UART_7BIT_BYTE,UartControllerStopBit.UART_ONE_STOP_BIT)

{'module': 1,
 'opcode': 0,
 'message': 'UART CONTROLLER SET PARAMETERS requests success'}

>> New message from SUPERNOVA:
{'id': 28, 'command': 66, 'name': ' UART CONTROLLER SET PARAMETERS', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}


In [37]:
supernova.uartControllerSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE requests success'}

>> New message from SUPERNOVA:
{'id': 29, 'command': 67, 'name': ' UART CONTROLLER SEND MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR'}
>> New message from SUPERNOVA:
{'id': 0, 'command': 68, 'name': ' UART CONTROLLER RECEIVE MESSAGE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'UART_NO_ERROR', 'driver_error': 'NO_TRANSFER_ERROR', 'payload_length': 1024, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10